In [404]:
import requests
from lxml import html
import pandas as pd
import time

In [428]:
allzipcodes = pd.read_csv('us_postal_codes.csv')
# allzipcodes['Zip'] = allzipcodes['Zip'].apply(lambda x : int(x))
allzipcodes['Zip Code'] = allzipcodes['Zip Code'].apply(lambda x : str(x))
allzipcodes['Zip Code'] = allzipcodes['Zip Code'].apply(lambda x : '0' + x if len(x) == 4 else x)
allzipcodes['Zip Code'] = allzipcodes['Zip Code'].apply(lambda x : '00' + x if len(x) == 3 else x)
zipcodes = allzipcodes['Zip Code']

In [436]:
acsquery = ('https://api.census.gov/data/2013/acs5?get='
'B25119_001E,' #median household income
'B25008_001E,' #total population
'B01002_001E,' #median age
'B25008_002E'  #population owner occupied
'&for=zip%20code%20tabulation%20area:*')

In [438]:
page = requests.get(acsquery)

In [442]:
page[1]

TypeError: 'Response' object does not support indexing

In [433]:
renterrate = []
costofliving = []
population = []
medianhomeprice = []
medianage = []
medianincome = []
precessedzips = []
for i, code in enumerate(zipcodes):

    
    print("{} of {} zipcodes processed".format(i+1, len(zipcodes)), end = '\r')
    zipcode = str(code)
    url = 'http://www.city-data.com/zips/'+ zipcode + '.html'
    page = requests.get(url)

    #rental rate
    location = page.text.find('% of renters here:')
    section = page.text[location+50: location +79]
    start = section.find('</p>')
    end = section.find('%')
    renterrate.append(section[start+4:end])

    #cost of living
    location = page.text.find('cost of living index in zip code')
    section = page.text[location: location +100]
    start = section.find('</b>')
    end = section.find('<b>')
    costofliving.append(section[start+5:end])
    
    #population
    location = page.text.find('Zip code population in 2010:')
    section = page.text[location: location +100]
    start = section.find('</b>')
    end = section.find('<b>')
    print(section[start:end])

    #median home value
#     location = page.text.find('Estimated median house/condo value in 2015:')
#     section = page.text[location: location +100]
#     start = section.find('</b>')
#     end = section.find('<b>')
#     print(section[start:end]) 
    
    #median age
#     location = page.text.find('Median resident age:')
#     section = page.text[location: location +100]
#     start = section.find('</b>')
#     end = section.find('<b>')
#     print(section[start:end])
    
    #median income
#     location = page.text.find('Average Adjusted Gross Income:')
#     section = page.text[location: location +100]
#     start = section.find('</b>')
#     end = section.find('<b>')
#     print(section[start:end])
    

    precessedzips.append(code)
    if i+1 % 450 == 0:
        time.sleep(3600)    


df = pd.DataFrame()
df['zip'] = precessedzips
df['renterrate%'] = renterrate
df['costofliving'] = costofliving
df['population'] = population
df['medianhomeprice'] = medianhomeprice
df['medianage'] = medianage
df['medianincome'] = medianincome

df.to_csv('Scraped_Other_Data.csv', index = False, header = True)

KeyboardInterrupt: 

In [391]:
df['renterrate%'] = df['renterrate%'].apply(lambda x : x if str.isdigit(x) else 'error')
df['costofliving'] = df['costofliving'].apply(lambda x : x if x else 'error')